In [27]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [28]:
from activ import load_data, data_normalization
import os
from activ.readfile import TrackTBIFile
import h5py
import numpy as np
from os.path import dirname, join
from IPython.display import display, Latex, Markdown
from scipy.stats import norm, sem
import scipy.optimize as spo
from activ import load_data

In [29]:
from ipywidgets import interact
import ipywidgets as widgets
from bokeh.layouts import row, widgetbox, column
from bokeh.models import Select, ColumnDataSource
from bokeh.palettes import Greys8
from bokeh.plotting import curdoc, figure

In [30]:
nmf = load_data(uoinmf=True)
raw = load_data(uoinmf=False)

In [42]:
bm = nmf.biomarkers
bm_features = nmf.biomarker_features
oc = nmf.outcomes
oc_features = nmf.outcome_features
features = np.concatenate((bm_features, oc_features)).tolist()

In [32]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components=1)
bm_cca, oc_cca = cca.fit_transform(bm, oc)

In [33]:
import pandas as pd
columns = ['bm_cca', 'oc_cca', 'gose_3', 'gose_6']
df = pd.DataFrame(columns=columns)

In [34]:
bm_cca = bm_cca.ravel()
oc_cca = oc_cca.ravel()

In [35]:
df['bm_cca'] = bm_cca
df['oc_cca'] = oc_cca
df['gose_3'] = raw.outcomes[:,4].astype(int)
df['gose_6'] = raw.outcomes[:,32].astype(int)

In [36]:
df = df.copy()

In [37]:
SIZES = list(range(6, 22, 3))
COLORS = Greys8[::-1]
N_SIZES = len(SIZES)
N_COLORS = len(COLORS)
scores = ['3', '4', '5', '6', '7', '8']

In [38]:
xs = df['bm_cca'].values
xs3 = df[df['gose_3']==3]
xs3_new = xs3['bm_cca'].values
print(xs3)
print(xs3_new)

       bm_cca    oc_cca  gose_3  gose_6
28  -2.212360 -2.037118       3       3
42  -2.097405 -3.201969       3       3
45  -1.752672 -1.833708       3       5
68  -1.456278 -3.053197       3       4
138  0.033538 -0.627626       3       4
[-2.2123603  -2.0974046  -1.75267161 -1.4562776   0.03353805]


In [12]:
def make_plot(df):
    xs = df['bm_cca'].values
    ys = df['oc_cca'].values
    
    p = figure(plot_height=600, plot_width=600, tools='pan,box_zoom,hover,reset')
    p.xaxis.axis_label = 'Biomarkers'
    p.yaxis.axis_label = 'Outcomes'
    
    if gose3.value == 'All':
        xs3 = xs
        ys3 = ys
        size_data = 'gose_3'
        
    elif gose3.value == 'None':
        xs3 = pd.DataFrame()
        ys3 = pd.DataFrame()
    else:
        xs3 = df[df['gose_3']==int(gose3.value)]
        xs3 = xs3['bm_cca'].values
        ys3 = df[df['gose_3']==int(gose3.value)]
        ys3 = ys3['oc_cca'].values
    
    if gose6.value == 'All':
        xs6 = xs
        ys6 = ys
        size_data = 'gose_6'
     
    elif gose6.value == 'None':
        xs6 = pd.DataFrame()
        ys6 = pd.DataFrame()
    else:
        xs6 = df[df['gose_6']==int(gose6.value)]
        xs6 = xs6['bm_cca'].values
        ys6 = df[df['gose_6']==int(gose6.value)]
        ys6 = ys6['oc_cca'].values
    
    sz = 9
    if size.value != 'No':
        size_groups = pd.Categorical(df[size_data])
        sz = [SIZES[xx] for xx in size_groups.codes]
        
    c3 = "#31AADE"
    c6 = "yellow"
    if color.value != 'No':
        color_groups_3 = pd.Categorical(df['gose_3'])
        c3 = [COLORS[xx] for xx in color_groups_3.codes]
        
        color_groups_6 = pd.Categorical(df['gose_6'])
        c6 = [COLORS[xx] for xx in color_groups_6.codes]
    
    p.circle(x=xs3, y=ys3, color=c3, size=sz, alpha=0.6, hover_color='white', hover_alpha=0.5)
    p.circle(x=xs6, y=ys6, color=c6, size=sz, alpha=0.6, hover_color='white', hover_alpha=0.5)
    return p 

In [43]:
def make_plot2(df):
    xs = df['bm_cca'].values
    ys = df['oc_cca'].values
    
    p = figure(plot_height=600, plot_width=600, tools='pan,box_zoom,hover,reset')
    p.xaxis.axis_label = 'Biomarkers'
    p.yaxis.axis_label = 'Outcomes'
    return p

In [44]:
def update(attr, old, new):
    layout.children[1] = make_plot2(df)
    push_notebook()

In [14]:
from bokeh.resources import CDN
from bokeh.embed import file_html

value3 = 'All'
value6 = 'All'
value_sz = 'No'
value_col = 'No'

gose3 = Select(title='GOSE 3 mo', value=value3, options=['All', 'None']+scores)
gose6 = Select(title='GOSE 6 mo', value=value6, options=['All', 'None']+scores)
size = Select(title='Size', value=value_sz, options=['No', 'Yes'])
color = Select(title='Color', value=value_col, options=['No', 'Yes'])

gose3.on_change('value', update)
gose6.on_change('value', update)
size.on_change('value', update)
color.on_change('value', update)

controls = widgetbox([gose3, gose6, color, size], width=200)
layout = row(controls, make_plot(df))

curdoc().add_root(layout)
h = show(layout, notebook_handle=True)
interact(update, handle=h)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



ValueError: cannot find widget or abbreviation for argument: 'attr'

In [45]:
# plot2

from bokeh.resources import CDN
from bokeh.embed import file_html

value_sz = 'No'
value_col = 'No'

size = Select(title='Size', value=value_sz, options=features)
color = Select(title='Color', value=value_col, options=features)

size.on_change('value', update)
color.on_change('value', update)

controls = widgetbox([color, size], width=200)
layout = row(controls, make_plot(df))

curdoc().add_root(layout)
h = show(layout, notebook_handle=True)
interact(update, handle=h)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



ValueError: cannot find widget or abbreviation for argument: 'attr'